In [14]:
!pip install torchtext==0.17

  Using cached torchtext-0.17.0-cp310-cp310-manylinux1_x86_64.whl.metadata (7.6 kB)
  Using cached torch-2.2.0-cp310-cp310-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached torchdata-0.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_6

Checklist:
- download the datasets
- tokenize and add token columns to the dataset
- build vocab from iterator
- convert tokens to ids and add new cols to the dataset
- data loaders and collate fn
- encoder
-decoder

In [12]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [4]:
dataset = datasets.load_dataset("bentrevett/multi30k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/4.60M [00:00<?, ?B/s]

val.jsonl:   0%|          | 0.00/164k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/156k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/29000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1014 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
#split the data

train_data,val_data,test_data=dataset["train"],dataset["validation"],dataset["test"]
type(train_data)

datasets.arrow_dataset.Dataset

#### Exploring train_data

In [6]:
type(train_data)

datasets.arrow_dataset.Dataset

In [7]:
train_data.shape, train_data[0:5]

((29000, 2),
 {'en': ['Two young, White males are outside near many bushes.',
   'Several men in hard hats are operating a giant pulley system.',
   'A little girl climbing into a wooden playhouse.',
   'A man in a blue shirt is standing on a ladder cleaning a window.',
   'Two men are at the stove preparing food.'],
  'de': ['Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
   'Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.',
   'Ein kleines Mädchen klettert in ein Spielhaus aus Holz.',
   'Ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster.',
   'Zwei Männer stehen am Herd und bereiten Essen zu.']})

It has 2 keys 'en' for english and 'de' for german

In [8]:
for i,elem in enumerate(train_data):
  print(elem)
  if i==3:
    break


{'en': 'Two young, White males are outside near many bushes.', 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}
{'en': 'Several men in hard hats are operating a giant pulley system.', 'de': 'Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.'}
{'en': 'A little girl climbing into a wooden playhouse.', 'de': 'Ein kleines Mädchen klettert in ein Spielhaus aus Holz.'}
{'en': 'A man in a blue shirt is standing on a ladder cleaning a window.', 'de': 'Ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster.'}


Every element in the data is a dictionary, with keys en and de

### tokenization

In [9]:
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 84.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
en_nlp=spacy.load("en_core_web_sm")
de_nlp=spacy.load("de_core_news_sm")

In [11]:
sent="How you do everything is how you do anything"
[[token.text,token.pos_,token.dep] for token in en_nlp.tokenizer(sent)]

[['How', '', 0],
 ['you', '', 0],
 ['do', '', 0],
 ['everything', '', 0],
 ['is', '', 0],
 ['how', '', 0],
 ['you', '', 0],
 ['do', '', 0],
 ['anything', '', 0]]

In [12]:
# aim to tokenize each example of the train,val and test data

def tokenizer(example,en_nlp,de_nlp,max_length,lower,sos_token,eos_token):
  #since example will be a dict with keys en and de
  en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
  de_tokens=[token.text for token in de_nlp.tokenizer(example["de"])][:max_length]

  if lower:
    en_tokens=[token.lower() for token in en_tokens]
    de_tokens=[token.lower() for token in de_tokens]

  #add sos and eos
  en_tokens=[sos_token] + en_tokens + [eos_token]
  de_tokens=[sos_token] + en_tokens + [eos_token]

  #intention: to create new keys in a singular example dictionary, which act like columns
  return {"en_tokens":en_tokens, "de_tokens":de_tokens}

max_length=1000
lower=True
sos_token="<sos>"
eos_token="<eos>"

fn_kwargs={
    "en_nlp":en_nlp,
    "de_nlp":de_nlp,
    "max_length":max_length,
    "lower":lower,
    "sos_token":sos_token,
    "eos_token":eos_token
}

train_data=train_data.map(tokenizer,fn_kwargs=fn_kwargs)
val_data=val_data.map(tokenizer,fn_kwargs=fn_kwargs)
test_data=test_data.map(tokenizer,fn_kwargs=fn_kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### building vocabulary

https://pytorch.org/text/stable/vocab.html#build-vocab-from-iterator

In [13]:
type(train_data["en_tokens"])

list

In [14]:
min_freq=2
unk_token="<unk>"
pad_token="<pad>"
specials=[
    unk_token,
    pad_token,
    sos_token,
    eos_token
]

en_vocab=torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq,
    specials
)

de_vocab=torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq,
    specials
)

In [15]:
en_vocab[unk_token]

0

In [16]:
#setting a default index for tokens not available in the vocabulary
en_vocab.set_default_index(en_vocab[unk_token])
de_vocab.set_default_index(de_vocab[unk_token])

In [17]:
#numericalizing the examples and creating 2 new keys

def numericalize_example(example,en_vocab,de_vocab):
  en_ids=en_vocab.lookup_indices(example["en_tokens"])
  de_ids=de_vocab.lookup_indices(example["de_tokens"])
  return {"en_ids":en_ids,"de_ids":de_ids}

fn_kwargs={"en_vocab":en_vocab,"de_vocab":de_vocab}
train_data=train_data.map(numericalize_example,fn_kwargs=fn_kwargs)
val_data=val_data.map(numericalize_example,fn_kwargs=fn_kwargs)
test_data=test_data.map(numericalize_example,fn_kwargs=fn_kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [18]:
#token ids should be of type torch tensor
data_type="torch"
format_columns=["en_ids","de_ids"]
train_data=train_data.with_format(
    type=data_type,columns=format_columns,output_all_columns=True
)
val_data=val_data.with_format(
    type=data_type,columns=format_columns,output_all_columns=True)
test_data=test_data.with_format(
    type=data_type,columns=format_columns,output_all_columns=True)

### Data loaders

In [19]:
#use a collate fn to pad the examples, since they may be of varying length

def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn


def get_data_loader(dataset,batch_size,pad_index,shuffle=False):
  collate_fn=get_collate_fn(pad_index)
  data_loader=torch.utils.data.DataLoader(
      dataset,
      batch_size=batch_size,
      shuffle=shuffle,
      collate_fn=collate_fn
  )
  return data_loader

batch_size=128
pad_index=en_vocab[pad_token]
train_data_loader=get_data_loader(train_data,batch_size,pad_index)
val_data_loader = get_data_loader(val_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

## Model

### Encoder

In [20]:
class Encoder(nn.Module):
  def __init__(self,input_dim,embedding_dim,hidden_dim,dropout):
    super().__init__()
    self.hidden_dim=hidden_dim
    self.embedding=nn.Embedding(input_dim,embedding_dim)
    #GRU was introduced in this paper
    self.rnn=nn.GRU(embedding_dim,hidden_dim)
    self.dropout=nn.Dropout(dropout)


  def forward(self,src):
    #src=[src_length, batch_size]
    embedded=self.dropout(self.embedding(src))
    #embedded=[src_length, batch_size, embedding_dim]
    #outputs always from the top hidden layer
    outputs,hidden=self.rnn(embedded)
    #outputs = [src_length, batch_size, hidden_dim*n_directions]
    #hidden=[n_layers*n_directions,batch_size,hidden_dim]


    return hidden

### Decoder

In [21]:
class Decoder(nn.Module):
  def __init__(self,output_dim,embedding_dim,hidden_dim,dropout):
    super().__init__()
    self.hidden_dim=hidden_dim
    self.output_dim=output_dim
    self.embedding=nn.Embedding(output_dim,embedding_dim)
    #addition of the input+ hidden vector
    self.rnn=nn.GRU(embedding_dim + hidden_dim,hidden_dim)
    self.fc_out=nn.Linear(embedding_dim + hidden_dim*2,output_dim)
    self.dropout=nn.Dropout(dropout)

  def forward(self,input,hidden,context):
    #input=[batch_size]
    #hidden=[n_layers*n_directions,batch_size,hidden_dim]
    #context=[n_layers*n_directions,batch_size,hidden_dim]

    input=input.unsqueeze(0)
    #input=[1,batch_size]

    embedded=self.dropout(self.embedding(input))
    emb_con=torch.cat((embedded,context),dim=2)
    output,hidden=self.rnn(emb_con,hidden)
    output=torch.cat(
        (embedded.squeeze(0),hidden.squeeze(0),context.squeeze(0)),dim=1
    )

    prediction=self.fc_out(output)
    return prediction,hidden



### Seq2Seq

In [22]:
class Seq2Seq(nn.Module):

  def __init__(self,encoder,decoder,device):
    super().__init__()
    self.encoder=encoder
    self.decoder=decoder
    self.device=device

  def forward(self,src,trg,teacher_forcing_ratio):
    #src= [src_len, batch_size]
    #trg=[trg len, batch_size]

    batch_size=src.shape[1]
    trg_length=trg.shape[0]

    trg_vocab_size=self.decoder.output_dim

    outputs=torch.zeros(trg_length,batch_size,trg_vocab_size).to(self.device)
    context=self.encoder(src)
    hidden=context #since the initial hidden state is the context
    #first input to the decoder is <sos>
    input=trg[0,:]

    for t in range(1,trg_length):

      output,hidden=self.decoder(input,hidden,context)
      outputs[t]=output
      teacher_force=random.random() < teacher_forcing_ratio

      top1=output.argmax(1)
      input=trg[t] if teacher_force else top1
      #input=[batch_size]

    return outputs

In [23]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

From the paper "All the weight parameters in the RNN Encoder–
Decoder were initialized by sampling from an
isotropic zero-mean (white) Gaussian distribution
with its standard deviation fixed to 0.01, except
for the recurrent weight parameters."

In [24]:
def init_weights(model):
  for name,params in model.named_parameters():
    nn.init.normal_(params.data,mean=0,std=0.01)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [25]:
optimizer = optim.Adam(model.parameters())

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

### training

In [26]:
def train_model(model,data_loader,optimizer,criterion,teacher_forcing_ratio,clip,device):
  model.train()
  epoch_loss=0
  for i,batch in tqdm.tqdm(enumerate(data_loader)):
    src=batch["de_ids"].to(device)
    trg=batch["en_ids"].to(device)

    optimizer.zero_grad()
    output=model(src,trg,teacher_forcing_ratio)
    output_dim=output.shape[-1]
    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)
    loss = criterion(output, trg)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss / len(data_loader)

In [27]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)

            output = model(src, trg, 0)

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [28]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [29]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_model(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        val_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut2-model.pt")
    print(f"Train Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"Valid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

  0%|          | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:01,  1.11s/it]
2it [00:01,  1.57it/s]
3it [00:01,  2.21it/s]
4it [00:01,  2.89it/s]
5it [00:02,  3.40it/s]
6it [00:02,  4.01it/s]
7it [00:02,  4.43it/s]
8it [00:02,  4.89it/s]
9it [00:02,  5.17it/s]
10it [00:02,  5.21it/s]
11it [00:03,  5.22it/s]
12it [00:03,  5.05it/s]
13it [00:03,  4.79it/s]
14it [00:03,  4.94it/s]
15it [00:03,  5.02it/s]
16it [00:04,  4.80it/s]
17it [00:04,  4.32it/s]
18it [00:04,  4.28it/s]
19it [00:04,  4.43it/s]
20it [00:05,  4.57it/s]
21it [00:05,  4.69it/s]
22it [00:05,  4.80it/s]
23it [00:05,  4.41it/s]
24it [00:05,  4.42it/s]
25it [00:06,  4.35it/s]
26it [00:06,  4.31it/s]
27it [00:06,  4.21it/s]
28it [00:06,  4.55it/s]
29it [00:07,  4.87it/s]
30it [00:07,  4.87it/s]
31it [00:07,  4.93it/s]
32it [00:07,  4.96it/s]
33it [00:07,  5.04it/s]
34it [00:07,  5.37it/s]
35it [00:08,  5.52it/s]
36it [00:08,  5.29it/s]
37it [00:08,  5.58it/s]
38it [00:08,  5.47it/s]
39it [00:08,  5.42it/s]
40it [00:09,  5

Train Loss:   4.648 | Train PPL: 104.395
Valid Loss:   6.532 | Valid PPL: 686.469



0it [00:00, ?it/s]
1it [00:00,  6.38it/s]
2it [00:00,  4.78it/s]
3it [00:00,  4.35it/s]
4it [00:00,  4.57it/s]
5it [00:01,  4.69it/s]
6it [00:01,  5.00it/s]
7it [00:01,  5.13it/s]
8it [00:01,  5.38it/s]
9it [00:01,  5.46it/s]
10it [00:01,  5.42it/s]
11it [00:02,  5.40it/s]
12it [00:02,  5.08it/s]
13it [00:02,  4.80it/s]
14it [00:02,  4.91it/s]
15it [00:02,  5.02it/s]
16it [00:03,  5.07it/s]
17it [00:03,  4.67it/s]
18it [00:03,  4.67it/s]
19it [00:03,  4.85it/s]
20it [00:04,  5.10it/s]
21it [00:04,  5.26it/s]
22it [00:04,  5.43it/s]
23it [00:04,  5.00it/s]
24it [00:04,  5.14it/s]
25it [00:04,  5.03it/s]
26it [00:05,  5.00it/s]
27it [00:05,  4.87it/s]
28it [00:05,  5.05it/s]
29it [00:05,  5.23it/s]
30it [00:05,  5.11it/s]
31it [00:06,  5.10it/s]
32it [00:06,  5.03it/s]
33it [00:06,  5.03it/s]
34it [00:06,  5.15it/s]
35it [00:06,  5.06it/s]
36it [00:07,  4.66it/s]
37it [00:07,  4.82it/s]
38it [00:07,  4.69it/s]
39it [00:07,  4.58it/s]
40it [00:08,  4.57it/s]
41it [00:08,  4.77it/s]
42it 

Train Loss:   3.611 | Train PPL:  37.011
Valid Loss:   6.689 | Valid PPL: 803.357



0it [00:00, ?it/s]
1it [00:00,  6.38it/s]
2it [00:00,  4.37it/s]
3it [00:00,  4.20it/s]
4it [00:00,  4.56it/s]
5it [00:01,  4.60it/s]
6it [00:01,  4.97it/s]
7it [00:01,  5.03it/s]
8it [00:01,  5.28it/s]
9it [00:01,  5.38it/s]
10it [00:01,  5.32it/s]
11it [00:02,  5.28it/s]
12it [00:02,  4.85it/s]
13it [00:02,  4.59it/s]
14it [00:02,  4.72it/s]
15it [00:03,  4.85it/s]
16it [00:03,  4.88it/s]
17it [00:03,  4.43it/s]
18it [00:03,  4.50it/s]
19it [00:03,  4.66it/s]
20it [00:04,  4.91it/s]
21it [00:04,  5.09it/s]
22it [00:04,  5.28it/s]
23it [00:04,  4.92it/s]
24it [00:04,  5.04it/s]
25it [00:05,  5.00it/s]
26it [00:05,  4.94it/s]
27it [00:05,  4.80it/s]
28it [00:05,  4.96it/s]
29it [00:05,  5.23it/s]
30it [00:06,  5.06it/s]
31it [00:06,  5.04it/s]
32it [00:06,  4.93it/s]
33it [00:06,  4.97it/s]
34it [00:06,  5.25it/s]
35it [00:07,  5.34it/s]
36it [00:07,  5.10it/s]
37it [00:07,  5.28it/s]
38it [00:07,  5.22it/s]
39it [00:07,  5.15it/s]
40it [00:08,  5.21it/s]
41it [00:08,  5.52it/s]
42it 

...

Train Loss:   0.828 | Train PPL:   2.288
Valid Loss:   3.641 | Valid PPL:  38.113



0it [00:00, ?it/s]
1it [00:00,  6.20it/s]
2it [00:00,  4.64it/s]
3it [00:00,  4.35it/s]
4it [00:00,  4.68it/s]
5it [00:01,  4.61it/s]
6it [00:01,  4.88it/s]
7it [00:01,  5.04it/s]
8it [00:01,  5.29it/s]
9it [00:01,  5.37it/s]
10it [00:01,  5.31it/s]
11it [00:02,  5.28it/s]
12it [00:02,  4.97it/s]
13it [00:02,  4.70it/s]
14it [00:02,  4.81it/s]
15it [00:03,  4.93it/s]
16it [00:03,  4.97it/s]
17it [00:03,  4.55it/s]
18it [00:03,  4.55it/s]
19it [00:03,  4.74it/s]
20it [00:04,  4.96it/s]
21it [00:04,  5.14it/s]
22it [00:04,  5.35it/s]
23it [00:04,  4.97it/s]
24it [00:04,  5.09it/s]
25it [00:05,  4.77it/s]
26it [00:05,  4.59it/s]
27it [00:05,  4.42it/s]
28it [00:05,  4.48it/s]
29it [00:05,  4.71it/s]
30it [00:06,  4.45it/s]
31it [00:06,  4.40it/s]
32it [00:06,  4.23it/s]
33it [00:06,  4.26it/s]
34it [00:07,  4.49it/s]
35it [00:07,  4.57it/s]
36it [00:07,  4.34it/s]
37it [00:07,  4.57it/s]
38it [00:08,  4.62it/s]
39it [00:08,  4.74it/s]
40it [00:08,  4.90it/s]
41it [00:08,  5.27it/s]
42it 

Train Loss:   0.647 | Train PPL:   1.911
Valid Loss:   3.989 | Valid PPL:  54.016



0it [00:00, ?it/s]
1it [00:00,  5.80it/s]
2it [00:00,  4.46it/s]
3it [00:00,  4.27it/s]
4it [00:00,  4.58it/s]
5it [00:01,  4.62it/s]
6it [00:01,  4.97it/s]
7it [00:01,  5.09it/s]
8it [00:01,  5.38it/s]
9it [00:01,  5.45it/s]
10it [00:01,  5.38it/s]
11it [00:02,  5.27it/s]
12it [00:02,  4.88it/s]
13it [00:02,  4.65it/s]
14it [00:02,  4.77it/s]
15it [00:03,  4.89it/s]
16it [00:03,  4.92it/s]
17it [00:03,  4.47it/s]
18it [00:03,  4.55it/s]
19it [00:03,  4.74it/s]
20it [00:04,  4.95it/s]
21it [00:04,  5.15it/s]
22it [00:04,  5.24it/s]
23it [00:04,  4.92it/s]
24it [00:04,  5.04it/s]
25it [00:05,  4.98it/s]
26it [00:05,  4.94it/s]
27it [00:05,  4.78it/s]
28it [00:05,  4.95it/s]
29it [00:05,  5.25it/s]
30it [00:06,  5.09it/s]
31it [00:06,  5.06it/s]
32it [00:06,  4.93it/s]
33it [00:06,  4.96it/s]
34it [00:06,  5.26it/s]
35it [00:07,  5.11it/s]
36it [00:07,  4.63it/s]
37it [00:07,  4.65it/s]
38it [00:07,  4.55it/s]
39it [00:07,  4.56it/s]
40it [00:08,  4.51it/s]
41it [00:08,  4.74it/s]
42it 

Train Loss:   0.508 | Train PPL:   1.662
Valid Loss:   3.752 | Valid PPL:  42.594


In [30]:
device

device(type='cuda')